<h2> Distributed Analysis Using Apache Spark Standalone Cluster Deployed On Docker Containers </h2>

<h4>Avinash Pasupulate</h4>
<br><i>avinash.pasupulate(at)gmail(dot)com
<br>https://github.com/avinashpasupulate
<br>https://www.linkedin.com/in/avinashpasupulate/
</i>

In [ ]:
from pyspark.sql import SparkSession

In [30]:
#installing numpy
!pip install numpy

import numpy as np

In [31]:
#creating a spark session
spark = SparkSession\
                    .builder\
                    .appName('pyspark_01')\
                    .master('spark://spark-master:7077')\
                    .config\
                        ('spark.submit.deployMode', 'cluster')\
                    .getOrCreate()


In [32]:
#downloading the raw data to the container
!curl -O https://raw.githubusercontent.com/avinashpasupulate/pyspark_cluster_setup_and_regression/master/SMSSpamCollection

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  466k  100  466k    0     0   288k      0  0:00:01  0:00:01 --:--:--  289k


In [33]:
import os
os.listdir(os.getcwd())

['.ipynb_checkpoints',
 'pyspark_01.ipynb',
 'spark-warehouse',
 'SMSSpamCollection']

In [34]:
#importing raw data
df = spark.read.format('csv').load('SMSSpamCollection', inferSchema = True, sep = '\t')

In [35]:
df.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [36]:
#renaming columns 
df = df.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [37]:
from pyspark.sql.functions import length

In [38]:
#including the length of text in each row on the dataframe
df = df.withColumn('length', length(df['text']))

In [39]:
df.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [40]:
#getting average length of text in each class
df.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.48663212435233|
| spam|138.6706827309237|
+-----+-----------------+



In [41]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [42]:
#processign and cleaning text data
tokenizer = Tokenizer(inputCol = 'text', outputCol = 'token_text')
stopword_remove = StopWordsRemover(inputCol = 'token_text', outputCol = 'stop_token')
count_vec = CountVectorizer(inputCol = 'stop_token', outputCol = 'count_vec')
idf = IDF(inputCol = 'count_vec', outputCol = 'tf_idf')
class_to_numeric = StringIndexer(inputCol = 'class', outputCol = 'label')

In [43]:
from pyspark.ml.feature import VectorAssembler

In [44]:
assembler = VectorAssembler(inputCols = ['tf_idf', 'length'], outputCol='features')

In [45]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline

In [46]:
nb = NaiveBayes()

In [47]:
#creating a processing pipeline with the stages to apply on the data
proc_pipe = Pipeline(stages = [tokenizer, stopword_remove, count_vec, idf, class_to_numeric, assembler])

In [48]:
clean = proc_pipe.fit(df)

In [49]:
clean_data = clean.transform(df)

In [50]:
clean_data = clean_data.select(['label', 'features'])

In [51]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13461,[8,12,33,6...|
|  0.0|(13461,[0,26,308,...|
|  1.0|(13461,[2,14,20,3...|
|  0.0|(13461,[0,73,84,1...|
|  0.0|(13461,[36,39,140...|
|  1.0|(13461,[11,57,62,...|
|  0.0|(13461,[11,55,108...|
|  0.0|(13461,[133,195,4...|
|  1.0|(13461,[1,50,124,...|
|  1.0|(13461,[0,1,14,29...|
|  0.0|(13461,[5,19,36,4...|
|  1.0|(13461,[9,18,40,9...|
|  1.0|(13461,[14,32,50,...|
|  0.0|(13461,[42,99,101...|
|  0.0|(13461,[567,1744,...|
|  1.0|(13461,[32,113,11...|
|  0.0|(13461,[86,224,37...|
|  0.0|(13461,[0,2,52,13...|
|  0.0|(13461,[0,77,107,...|
|  1.0|(13461,[4,32,35,6...|
+-----+--------------------+
only showing top 20 rows



In [52]:
#separating data into training and testing dataset
train_data, test_data = clean_data.randomSplit([0.7, 0.3])

In [53]:
#using Naive bayes classifier on the training data
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes()

In [54]:
spam_classifier = nb.fit(train_data)

In [55]:
test_results = spam_classifier.transform(test_data)

In [57]:
#evaluating model fit and accuracy
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
eval_acc =  MulticlassClassificationEvaluator()
acc = eval_acc.evaluate(test_results)

In [58]:
#accuracy score
acc

0.9217443249398358